In [1]:
import spacy
nlp=spacy.load('en_core_web_sm')


In [2]:
doc=nlp(u'"Management is doing things right;leardership is doing the right things." -Peter Drucker')
for sent in doc.sents:
    print(sent)

"Management is doing things right;leardership is doing the right things."
-Peter Drucker


In [20]:
from spacy import Language
@Language.factory("custom_boundaries")
def set_custom_boundaries(nlp, name):
    def custom_boundaries(doc):
        for token in doc[:-1]:
            if token.text == ';':
                doc[token.i + 1].is_sent_start = True
        return doc
    return custom_boundaries

ValueError: [E004] Can't set up pipeline component: a factory for 'custom_boundaries' already exists. Existing factory: <function set_custom_boundaries at 0x000001D35066FB00>. New factory: <function set_custom_boundaries at 0x000001D350B0D1C0>

In [4]:
# nlp.add_pipe("custom_boundaries", before="parser")
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [5]:
doc2=nlp(u'"Management is doing things right ; leardership is doing the right things." -Peter Drucker')
for sent in doc2.sents:
    print(sent)

"Management is doing things right ; leardership is doing the right things."
-Peter Drucker


Changing the rules


In [8]:
mystring = u"This is a sentence. This is another.\n\nThis is a \nthird sentence."
doc=nlp(mystring)
for sent in doc.sents:
    print([token.text for token in sent])

['This', 'is', 'a', 'sentence', '.']
['This', 'is', 'another', '.', '\n\n']
['This', 'is', 'a', '\n', 'third', 'sentence', '.']


In [21]:
# CHANGING THE RULES
# from spacy.pipeline import SentenceSegmenter
@Language.factory("split_on_newlines")
def split_on_newlines(doc):
    start = 0
    seen_newline = False
    for word in doc:
        if seen_newline:
            yield doc[start:word.i]
            start = word.i
            seen_newline = False
        elif word.text.startswith('\n'): # handles multiple occurrences
            seen_newline = True
    yield doc[start:]      # handles the last group of tokens


from spacy.language import Language

def custom_sentence_segmenter(doc):
    for sent in split_on_newlines(doc):
        doc[sent.start:sent.end].is_parsed = False  # Set is_parsed to False for each split sentence
    return doc

ValueError: [E004] Can't set up pipeline component: a factory for 'split_on_newlines' already exists. Existing factory: <function set_custom_boundaries at 0x000001D35066F9C0>. New factory: <function split_on_newlines at 0x000001D350B0E980>

In [14]:
nlp.add_pipe(custom_sentence_segmenter, before="parser")


ValueError: [E966] `nlp.add_pipe` now takes the string name of the registered component factory, not a callable component. Expected string, but got <function custom_sentence_segmenter at 0x000001D34EE8FA60> (name: 'None').

- If you created your component with `nlp.create_pipe('name')`: remove nlp.create_pipe and call `nlp.add_pipe('name')` instead.

- If you passed in a component like `TextCategorizer()`: call `nlp.add_pipe` with the string name instead, e.g. `nlp.add_pipe('textcat')`.

- If you're using a custom component: Add the decorator `@Language.component` (for function components) or `@Language.factory` (for class components / factories) to your custom component and assign it a name, e.g. `@Language.component('your_name')`. You can then run `nlp.add_pipe('your_name')` to add it to the pipeline.